In [ ]:
import osrm
import numpy as np

In [ ]:
class gas_station:
    def __init__(self, id, lat, long):
        self.id = id
        self.lat = lat
        self.long = long

    def getId(self):
        return self._id

    def setId (self,id):
        self._id = id

    def getLat(self):
        return self._lat

    def setLat (self,lat):
        self._lat = lat

    def getLong(self):
        return self._long

    def setLong (self,long):
        self._long = long

    id = property(getId, setId)
    lat = property(getLat, setLat)
    long = property(getLong, setLong)


In [ ]:
# read gas stations from file
import csv
try:
    source = '/Users/ayeletarditi/Google Drive/Thesis/Gas stations/gas_stations_reduced_1.5km.txt'
    f = open(source, 'rt')
    reader = csv.reader(f, delimiter='\t')
    gas_stations = []
    for el in reader:
        g = gas_station(int(el[0]),float(el[1]),float(el[2]))
        gas_stations.append(g)
finally:
    f.close()
    #print(len(gas_stations))


    #output_file = open('/Users/ayeletarditi/Documents/Thesis/osrm_outputs/output.txt', 'w')
gas_stations_cor = ""
dest_stations = []
for index2 in range(len(gas_stations)):
    dest_stations.append(gas_stations[index2].id)
    gas_stations_cor += str(gas_stations[index2].long) +","+ str(gas_stations[index2].lat) + ";"
'remove the last ;'
gas_stations_cor = gas_stations_cor[:-1]
    #output_file.write(dest)



In [ ]:
# generate duration matrix between gas stations

import requests
try:
    output_file = open('/Users/ayeletarditi/Documents/Thesis/osrm_outputs/gas_stations_durations.txt', 'w')
    for index in range(len(gas_stations)):
            url_osrm = 'http://localhost:5000/table/v1/driving/' + gas_stations_cor + '?sources=' + str(index)
            osrm = requests.get(url_osrm)
            data = osrm.json()
            if 'message' not in data:
                durations  = data['durations']
                print(len(durations[0]))
                for x in range(len(durations[0])):
                    output_file.write(str(gas_stations[index].id) + '\t' + str(gas_stations[x].id) + '\t' + str(durations[0][x]) + '\n')
            else:
                print(str(data['message']))
finally:
    output_file.close()

In [ ]:
import pandas as pd


from decimal import *
import csv
getcontext().prec = 4
getcontext().rounding = ROUND_DOWN

In [ ]:
df = pd.read_csv('/Users/ayeletarditi/Google Drive/Thesis/code/stop_points.txt', delimiter = '\t', index_col='imsi')
df = df[~((df['start_lat']==df['end_lat']) & (df['start_long']==df['end_long']))]
df['start_lat_2_dec'] = df['start_lat'].apply(lambda x : Decimal(x)*1)
df['start_long_2_dec'] = df['start_long'].apply(lambda x : Decimal(x)*1)
df['end_lat_2_dec'] = df['end_lat'].apply(lambda x : Decimal(x)*1)
df['end_long_2_dec'] = df['end_long'].apply(lambda x : Decimal(x)*1)

In [ ]:
orig_df = df[['start_long_2_dec','start_lat_2_dec']].drop_duplicates()
dest_df = df[['end_long_2_dec','end_lat_2_dec']].drop_duplicates()

In [ ]:
import requests
def clac_durations_orig_gas_stations (gas_stations_cor, orig):
    try:
        output_file = open('/Users/ayeletarditi/Documents/Thesis/osrm_outputs/orig_gas_stations_durations.txt', 'a')
        cor = orig + ';' + gas_stations_cor
        #print(cor)
        url_osrm = 'http://localhost:5000/table/v1/driving/' + cor + '?sources=0'
        osrm = requests.get(url_osrm)
        data = osrm.json()
        if 'message' not in data:
            durations  = data['durations']
            #print(durations)
            for x in range(len(durations[0])):
                if x > 0:
                    output_file.write( orig + '\t' + str(gas_stations[x-1].id) + '\t' + str(durations[0][x]) + '\n')
        else:
            print(str(data['message']))
    finally:
        output_file.close()

In [ ]:
import requests
def clac_durations_gas_stations_dest (gas_stations_cor, dest):
    try:
        output_file = open('/Users/ayeletarditi/Documents/Thesis/osrm_outputs/gas_stations_dest_durations.txt', 'a')
        cor = dest + ';' + gas_stations_cor
        #print(cor)
        url_osrm = 'http://localhost:5000/table/v1/driving/' + cor + '?destinations=0'
        osrm = requests.get(url_osrm)
        data = osrm.json()
        if 'message' not in data:
            durations  = data['durations']
            #print(durations)
            for x in range(len(durations) ):
                if x > 0:
                    output_file.write( str(gas_stations[x-1].id)+'\t'+  dest + '\t' + str(durations[x][0]) + '\n')
        else:
            print(str(data['message']))
    finally:
        output_file.close()
        #return durations

In [ ]:
orig_df.apply(lambda x: clac_durations_orig_gas_stations(gas_stations_cor, '%.2f' % x.start_long_2_dec + ',' + '%.2f' % x.start_lat_2_dec),axis=1)
dest_df.apply(lambda x: clac_durations_gas_stations_dest(gas_stations_cor, '%.2f' % x.end_long_2_dec + ',' + '%.2f' % x.end_lat_2_dec),axis=1)

In [ ]:
orig_dest_df = df[['start_lat_2_dec','start_long_2_dec','end_lat_2_dec','end_long_2_dec']].drop_duplicates()

In [ ]:
orig_dest_df = orig_dest_df[~((orig_dest_df['start_lat_2_dec']==orig_dest_df['end_lat_2_dec']) & (orig_dest_df['start_long_2_dec']==orig_dest_df['end_long_2_dec']))]

In [ ]:
orig_dest_df.shape

In [ ]:
import requests
def clac_durations_orig_dest (orig, dest):
    try:
        output_file = open('/Users/ayeletarditi/Documents/Thesis/osrm_outputs/origin_dest_durations.txt', 'a')
        cor = orig + ';' + dest
        #print(cor)
        url_osrm = 'http://localhost:5000/table/v1/driving/' + cor + '?sources=0'
        osrm = requests.get(url_osrm)
        data = osrm.json()
        if 'message' not in data:
            durations  = data['durations']
            output_file.write( orig +'\t'+ dest + '\t' + str(durations[0][1]) + '\n')
        else:
            print(str(data['message']))
    finally:
        output_file.close()
        #return durations



In [ ]:
orig_dest_df.apply(lambda x: clac_durations_orig_dest( '%.2f' % x.start_long_2_dec + ',' + 
                                                       '%.2f' % x.start_lat_2_dec, '%.2f' % x.end_long_2_dec + ',' +  '%.2f' % x.end_lat_2_dec),axis=1)

In [ ]:
orig_dest_durations = pd.read_csv('/Users/ayeletarditi/Documents/Thesis/osrm_outputs/origin_dest_durations.txt',
                                  sep = '\t', header= None, names = ['orig','dest','duration'])

orig_dest_durations.duration = pd.to_numeric(orig_dest_durations.duration, errors='coerce')
orig_dest_durations = orig_dest_durations[orig_dest_durations.duration > 0]

In [ ]:
orig_gas_stations_durations = pd.read_csv('/Users/ayeletarditi/Documents/Thesis/osrm_outputs/orig_gas_stations_durations.txt', sep = '\t', header= None, names = ['orig', 'gas_station','duration'],
                                          dtype = {'orig':str,'gas_station':int, 'duration':float})

dest_gas_stations_durations = pd.read_csv('/Users/ayeletarditi/Documents/Thesis/osrm_outputs/gas_stations_dest_durations.txt', sep = '\t', header = None, names=['gas_station','dest','duration'],
                                          dtype={'dest': str, 'gas_station': int, 'duration': float})

orig_gas_stations_durations.index = orig_gas_stations_durations.apply(lambda x:
                                                                      x.orig.strip() + "$" + str(x.gas_station).strip(),axis = 1)

orig_gas_stations_durations_dict = orig_gas_stations_durations[['duration']].T.to_dict('list')   
  
dest_gas_stations_durations.index = dest_gas_stations_durations.apply(lambda x:
                                                                      x.dest.strip() + "$" + str(x.gas_station).strip(),axis = 1)
dest_gas_stations_durations_dict = dest_gas_stations_durations[['duration']].T.to_dict('list') 


In [ ]:
gas_stations_durations_matrix = pd.read_csv('/Users/ayeletarditi/Documents/Thesis/osrm_outputs/gas_stations_durations.txt', 
                        sep = '\t', header = None, names = ['source', 'destination', 'duration'], dtype = {'source':int, 'destination': str, 'duration':float})
#gas_stations_durations_matrix.source = pd.to_numeric(gas_stations_durations_matrix.source)
#gas_stations_durations_matrix.destination = pd.to_numeric(gas_stations_durations_matrix.destination)

gas_stations_durations_matrix.index = gas_stations_durations_matrix.apply(lambda x: str(x.source).strip() + "$" + str(x.destination).strip(),axis = 1)
gas_stations_durations_matrix_dict = gas_stations_durations_matrix[['duration']].T.to_dict('list')       

In [ ]:
# import itertools
# import operator
# def deduce_path (orig, dest, des_orig_d):
#     output_file = open('/Users/ayeletarditi/Documents/Thesis/osrm_outputs/chains_paths.txt', 'a')
#     candidates = {}
#     #print(orig, dest)
#     for sp in gas_stations:
#         orig_sp_d = orig_gas_stations_durations_dict[orig + '$' + str(sp.id)][0]
#         sp_dest_d = dest_gas_stations_durations_dict[dest + '$' + str(sp.id)][0]
#         #des_orig_d = orig_dest_df[(orig_dest_df.orig == orig) & (orig_dest_df.dest == dest)]['duration']
#         if (float(orig_sp_d) + float(sp_dest_d ) - des_orig_d) <= des_orig_d*0.01:
#             candidates[sp.id] = float(orig_sp_d)
#     sorted_candidates = sorted(candidates.items(), key=operator.itemgetter(1))
#     index = 0
#     longest_combs = (None, None)
#     print(orig, dest)
#     print(len(sorted_candidates))
#     for i in range(1, len(sorted_candidates)+1):
#         combs = []
#         #i-length tuples, in sorted order, no repeated elements
#         combs.extend(itertools.combinations(sorted_candidates, i))
#         for comb in combs:
#             inner_index = 0
#             duration = 0
#             for can in comb:
#                 if inner_index == 0:
#                     duration += can[1]
#                 if inner_index == len(comb) - 1:
#                     duration += dest_gas_stations_durations_dict[dest + '$' + str(can[0])][0]
#                 if inner_index > 0 and inner_index < len(comb):
#                     duration += gas_stations_durations_matrix_dict[str(int(comb[inner_index - 1][0]))
#                                                            + '$' + str(int(can[0]))][0]
#                 inner_index += 1
#             if (duration - des_orig_d <= des_orig_d*0.01):
#                 #print(len(comb))
#                 if not all(longest_combs):
#                     longest_combs = (comb,duration)
#                 else:
#                     if len(comb) > len(longest_combs[0]):
#                         longest_combs = (comb,duration)
#                     else:
#                         if len(comb) == len(longest_combs[0]) and duration < longest_combs[1]:
#                             longest_combs = (comb,duration)
#         print(len(longest_combs[0]) , i)
#         if len(longest_combs[0]) < i:
#             break
#     if all(longest_combs):
#         orig_lat_long = orig.split(",")
#         dest_lat_long = dest.split(",")
#         output_file.write(orig_lat_long[1].strip() + "," + orig_lat_long[0].strip()+ '\t' +
#                           dest_lat_long[1].strip() + "," + dest_lat_long[0].strip() + '\t' )
#         for sp in longest_combs[0]:
#             output_file.write(str(sp[0]) + ',')
#         output_file.write('\t' + str((longest_combs[1] - des_orig_d)/60) + "\n")

In [ ]:
#orig_dest_durations.apply(lambda x: deduce_path(x.orig, x.dest, x.duration), axis = 1)

In [ ]:
# df['start_date'] = pd.to_datetime(df['start_date'])

In [ ]:
#df['date'] = df['start_date'].apply(lambda x: x.date())

In [ ]:
#df['day'] = df['start_date'].apply(lambda x: x.date().day)

In [ ]:
df[~((df.start_lat_2_dec == df.end_lat_2_dec) & (df.start_long_2_dec == df.end_long_2_dec))].shape

In [ ]:
# import pandas as pd
# df = pd.read_csv('/Users/ayeletarditi/Documents/Thesis/osrm_outputs/Post Analysis outputs/df.txt', sep = '\t')

In [ ]:
# df[(df.start_lat_2_dec == 31.88) & (df.end_lat_2_dec == 31.88)]

In [ ]:
# df['day'] = pd.DatetimeIndex(df['start_date']).day

In [ ]:
# df['hour'] = pd.DatetimeIndex(df['start_date']).hour

In [ ]:
#df[(df.day == 1) ]

In [ ]:
#df['hour'] = df.start_date.apply(lambda x : x.hour)

In [ ]:
# df_cnt = df.groupby(by=['start_lat_2_dec','start_long_2_dec','end_lat_2_dec','end_long_2_dec','day','hour']).count()

In [ ]:
# df_cnt.reset_index(inplace=True)

In [ ]:
# df_cnt = df_cnt[~((df_cnt.start_lat_2_dec == df_cnt.end_lat_2_dec)
#                & (df_cnt.end_long_2_dec == df_cnt.end_long_2_dec))]

In [ ]:
#df_cnt['lamda_old'] = (df_cnt.day - 1)*23 + df_cnt.hour
# df_cnt['lamda'] = (df_cnt.day - 1)*24 + df_cnt.hour
#df_cnt.apply(lambda x : (x.day - 1)*23 + x.hour, axis = 1)

In [ ]:
# df_cnt[['day','hour']].drop_duplicates().to_csv('lambdas.csv', index = False)

In [ ]:
# res_test = pd.read_csv('chains_res_1_perc.csv')

In [ ]:
# res_test.head()

In [ ]:
# chains = pd.read_csv("/Users/ayeletarditi/Documents/Thesis/osrm_outputs/chains_paths_6_perc.txt", 
#                  sep = '\t', header= None, names = ['orig', 'dest', 'stations','time'])

In [ ]:
# chains['num_gas_stations'] = chains['stations'].apply(lambda x : len(x.split(',')) - 1)

In [ ]:
# import numpy as np
# df_pivot = df_cnt.pivot_table(values='start_date', index=['start_lat_2_dec','start_long_2_dec','end_lat_2_dec','end_long_2_dec'],
#     columns = 'lamda',aggfunc = np.sum)

In [ ]:
# df_pivot.reset_index(inplace=True)

In [ ]:
# df_pivot.start_lat_2_dec = np.round(df_pivot.start_lat_2_dec,2)
# df_pivot.start_long_2_dec = np.round(df_pivot.start_long_2_dec,2)
# df_pivot.end_lat_2_dec = np.round(df_pivot.end_lat_2_dec,2)
# df_pivot.end_long_2_dec = np.round(df_pivot.end_long_2_dec,2)

In [ ]:
# df_pivot.head()

In [ ]:
# df_pivot['start_lat_2_dec_str'] = pd.Series(["{0:.2f}".format(val ) for val in df_pivot.start_lat_2_dec])
# df_pivot['start_long_2_dec_str'] = pd.Series(["{0:.2f}".format(val ) for val in df_pivot.start_long_2_dec])

# df_pivot['end_lat_2_dec_str'] = pd.Series(["{0:.2f}".format(val ) for val in df_pivot.end_lat_2_dec])
# df_pivot['end_long_2_dec_str'] = pd.Series(["{0:.2f}".format(val ) for val in df_pivot.end_long_2_dec])

In [ ]:

df_pivot['orig'] = df_pivot.start_lat_2_dec_str.str.cat(df_pivot.start_long_2_dec_str, sep=',')

#df_pivot.apply(lambda x: str(x.start_lat_2_dec) + ',' + str(x.start_long_2_dec), axis=1)
df_pivot['dest'] = df_pivot.end_lat_2_dec_str.str.cat(df_pivot.end_long_2_dec_str, sep=',')
#df_pivot.apply(lambda x: str(x.end_lat_2_dec) + ',' + str(x.end_long_2_dec), axis = 1)

In [ ]:
df_merged = df_pivot.merge(chains, how = 'left', on=['orig','dest'])

In [ ]:
#tmp = df_merged[df_merged.num_gas_stations > 1].iloc[:,4:35]
#tmp.columns =['1','2','3','4','5','6','7','8','9','10','11','12',
                                      # '13','14','15','16','17','18','19','20','21','22','23'
                                       #,'24','25','26','27','28','orig','dest','stations'] 

In [ ]:
df_merged.columns = df_merged.columns.astype(str)

In [ ]:
df_merged = df_merged[df_merged.num_gas_stations > 1]

In [ ]:
res_df = df_merged.groupby(by='stations').sum()

In [ ]:
res_df.reset_index(inplace=True)

In [ ]:
res_df['stations'] = res_df['stations'].apply(lambda x : x[:-1])

In [ ]:
chains_tmp = chains[chains.num_gas_stations > 1].stations.drop_duplicates().apply(lambda x : x[:-1])

In [ ]:
res_df.stations.to_csv('test2', index=False)

In [ ]:
chains_tmp.drop_duplicates().to_csv('test1', index=False)

In [ ]:
df_pivot[(df_pivot.orig == "31.88,34.82") & (df_pivot.dest == "31.88,34.68")]

In [ ]:
res_df.to_csv('chains_res_6_perc.csv', index=False)

In [ ]:
res_df.shape

In [ ]:
df_merged['num_gas_stations'] = df_merged['stations'].apply(lambda x : len(x.split(',')) - 1)

In [ ]:
df['id'] = df.apply(lambda x: str(x.start_lat_2_dec) + ',' + str(x.start_long_2_dec)+',' + str(x.end_lat_2_dec) + ',' + str(x.end_long_2_dec), axis=1)

In [ ]:
df['orig'] = df.apply(lambda x: str(x.start_lat_2_dec) + ',' + str(x.start_long_2_dec), axis=1)
df['dest'] = df.apply(lambda x: str(x.end_lat_2_dec) + ',' + str(x.end_long_2_dec), axis = 1)

In [ ]:
df_chainse = df.merge(chains, how = 'left', 
                      left_on=['start_lat_2_dec','start_long_2_dec','end_lat_2_dec','end_long_2_dec'])